# 01 — Customer Profiles (Logistic Regression)

## Mục tiêu & phạm vi
- Xếp hạng khách hàng theo **xác suất “yes”** và rút **insight hành động** từ đặc điểm cá nhân.
- Biến sử dụng: `age, job, education, marital, housing, loan, default, y` *(không dùng `duration` khi train)*.

**Chiến lược đánh giá:** `cv5_test20`  
- Test 20% (stratified, seed=42) để đánh giá cuối.  
- Trên Train 80%, dùng Stratified KFold=5 để tuning Logistic (C ∈ {0.1, 1, 10}).

**Chuẩn chung:**  
- CSV đọc với `sep=';'`, map target `y: yes/no → 1/0`.  
- Giữ nhãn `"unknown"` như 1 giá trị hợp lệ.  
- Tránh leakage: **không dùng `duration`** trong mô hình triển khai.


## Phần 1: Chuẩn bị & Đọc dữ liệu
- Mục tiêu: nạp dữ liệu, chuẩn hoá target `y → y_bin`, chọn cột personal.
- Quy ước: không dùng `duration` khi train (tránh leakage).


In [7]:

SEED = 42
DATA_PATH = "../data/bank-additional/bank-additional-full.csv"

import numpy as np
import pandas as pd

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 120)


In [8]:
# Đọc CSV (sep=';') và chuẩn hoá target
df_raw = pd.read_csv(DATA_PATH, sep=';')
df = df_raw.copy()
df["y_bin"] = (df["y"].astype(str).str.strip().str.lower() == "yes").astype(int)

# Chọn cột personal cho phần này
PERSONAL_COLS = ["age","job","marital","education","default","housing","loan","y","y_bin"]
df_personal = df[PERSONAL_COLS].copy()

print("Columns:", list(df_personal.columns))
print("Shape:", df_personal.shape)
df_personal.head()

Columns: ['age', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'y', 'y_bin']
Shape: (41188, 9)


,age,job,marital,education,default,housing,loan,y,y_bin
0,56,housemaid,married,basic.4y,no,no,no,no,0
1,57,services,married,high.school,unknown,no,no,no,0
2,37,services,married,high.school,no,yes,no,no,0
3,40,admin.,married,basic.6y,no,no,no,no,0
4,56,services,married,high.school,no,no,yes,no,0


## Phần 2: Kiểm tra & Phân tích tần suất 
- Kiểm tra: tỉ lệ lớp, null, tỷ lệ "unknown", phạm vi tuổi.
- Tần suất/phân bố: đếm & tỷ lệ (%) cho các biến phân loại (job, education, marital, housing, loan, default, y).


In [9]:
# Tỉ lệ lớp
class_dist = df_personal["y_bin"].value_counts(normalize=True).rename({0:"no",1:"yes"}) * 100
print("Class distribution (%):\n", class_dist.round(2))

# Unique counts của biến phân loại & mô tả tuổi
cat_cols = ["job","marital","education","default","housing","loan","y"]
print("\nUnique counts (categorical):")
for c in cat_cols:
    print(f"{c:10s}: {df_personal[c].nunique()} labels")

print("\nAge describe:")
display(df_personal["age"].describe(percentiles=[.01,.05,.25,.5,.75,.95,.99]))


Class distribution (%):
 y_bin
no     88.73
yes    11.27
Name: proportion, dtype: float64

Unique counts (categorical):
job       : 12 labels
marital   : 4 labels
education : 8 labels
default   : 3 labels
housing   : 3 labels
loan      : 3 labels
y         : 2 labels

Age describe:


count    41188.00000
mean        40.02406
std         10.42125
min         17.00000
1%          23.00000
5%          26.00000
25%         32.00000
50%         38.00000
75%         47.00000
95%         58.00000
99%         71.00000
max         98.00000
Name: age, dtype: float64

In [10]:
# Null
nulls = df_personal.isna().sum().sort_values(ascending=False)
print("Nulls per column:\n", nulls)

# Tỷ lệ "unknown" theo cột danh mục
def unknown_rate(s: pd.Series) -> float:
    return (s.astype(str).str.strip().str.lower() == "unknown").mean() * 100
unk_summary = {c: round(unknown_rate(df_personal[c]), 2) for c in ["job","marital","education","default","housing","loan"]}
print("\n'unknown' rate (%) by column:\n", unk_summary)

# Phạm vi tuổi để flag outlier sớm
age = df_personal["age"]
age_bounds = {"min": int(age.min()), "p1": float(age.quantile(0.01)), "p99": float(age.quantile(0.99)), "max": int(age.max())}
print("\nAge bounds:", age_bounds)


Nulls per column:
 age          0
job          0
marital      0
education    0
default      0
housing      0
loan         0
y            0
y_bin        0
dtype: int64

'unknown' rate (%) by column:
 {'job': np.float64(0.8), 'marital': np.float64(0.19), 'education': np.float64(4.2), 'default': np.float64(20.87), 'housing': np.float64(2.4), 'loan': np.float64(2.4)}

Age bounds: {'min': 17, 'p1': 23.0, 'p99': 71.0, 'max': 98}


In [11]:
def freq_table(df_in: pd.DataFrame, col: str) -> pd.DataFrame:
    g = (
        df_in[col]
        .value_counts(dropna=False)
        .rename("count")
        .reset_index()
        .rename(columns={"index": col})
    )
    total = g["count"].sum()
    g["ratio (%)"] = (g["count"] / total * 100).round(2)
    return g

print("job — frequency:")
display(freq_table(df_personal, "job"))
print("\neducation — frequency:")
display(freq_table(df_personal, "education"))
print("\nmarital — frequency:")
display(freq_table(df_personal, "marital"))
print("\nhousing — frequency:")
display(freq_table(df_personal, "housing"))
print("\nloan — frequency:")
display(freq_table(df_personal, "loan"))
print("\ndefault — frequency:")
display(freq_table(df_personal, "default"))
print("\ny — frequency:")
display(freq_table(df_personal, "y"))


job — frequency:


,job,count,ratio (%)
0,admin.,10422,25.30
1,blue-collar,9254,22.47
2,technician,6743,16.37
3,services,3969,9.64
4,management,2924,7.10
5,retired,1720,4.18
6,entrepreneur,1456,3.54
7,self-employed,1421,3.45
8,housemaid,1060,2.57
9,unemployed,1014,2.46



education — frequency:


,education,count,ratio (%)
0,university.degree,12168,29.54
1,high.school,9515,23.10
2,basic.9y,6045,14.68
3,professional.course,5243,12.73
4,basic.4y,4176,10.14
5,basic.6y,2292,5.56
6,unknown,1731,4.20
7,illiterate,18,0.04



marital — frequency:


...

,loan,count,ratio (%)
0,no,33950,82.43
1,yes,6248,15.17
2,unknown,990,2.40



default — frequency:


,default,count,ratio (%)
0,no,32588,79.12
1,unknown,8597,20.87
2,yes,3,0.01



y — frequency:


,y,count,ratio (%)
0,no,36548,88.73
1,yes,4640,11.27


### Nhận xét tần suất & chất lượng dữ liệu (Customer Profiles)

**Tổng quan**
- 41,188 bản ghi, không có null. Lớp mục tiêu mất cân bằng: **yes 11.27%**, **no 88.73%**.

**Phân bố theo biến danh mục**
- **job (12 nhãn):** nhóm đông nhất: `admin.` (25.3%), `blue-collar` (22.47%), `technician` (16.37%).  
  Nhãn hiếm: `illiterate` không xuất hiện ở job; tuy nhiên `student` (2.12%), `unemployed` (2.46%), `housemaid` (2.57%), `unknown` (0.8%) là các nhóm nhỏ → giữ nguyên, chỉ cân nhắc gộp “other” nếu cần ổn định hệ số khi train.
- **education (8 nhãn):** `university.degree` (29.54%) và `high.school` (23.10%) chiếm hơn nửa tập;  
  `illiterate` cực hiếm (**0.04%**, 18 bản ghi) → có thể gộp với nhóm “basic.*” hoặc giữ riêng (tác động thấp).  
  `unknown` ở mức **4.2%** → giữ như nhãn hợp lệ để quan sát hành vi.
- **marital (4 nhãn):** `married` (60.52%) chiếm đa số; `single` (28.09%), `divorced` (11.20%), `unknown` rất ít (0.19%).  
  → Biến đủ sạch cho phân tích %yes theo trạng thái hôn nhân.
- **housing (3 nhãn):** `yes` (52.38%) nhỉnh hơn `no` (45.21%); `unknown` **2.4%** → giữ lại để đối chiếu.
- **loan (3 nhãn):** đa số `no` (82.43%); `yes` (15.17%); `unknown` **2.4%**.  
  → Kết hợp với `housing` để tìm nhóm “ít ràng buộc nợ”.
- **default (3 nhãn):** `no` (79.12%), **`unknown` 20.87%**, `yes` chỉ **3 bản ghi (0.01%)**.  
  → Phần “unknown” lớn **không phải lỗi** mà là thiếu thông tin nghiệp vụ; cần **giữ riêng**. Nhãn `default=yes` quá hiếm → mô hình khó học hệ số riêng, nên tập trung so sánh `no` vs `unknown`.
- **age (numeric):** min 17, p1=23, median=38, p75=47, p99=71, max 98 → phân bố hợp lý, không có outlier dị thường; sẽ **chuẩn hoá** khi vào Logistic và dùng **age band** cho EDA.

**Kết luận:** Dữ liệu sạch, chỉ mất cân bằng lớp và `default` có nhiều “unknown”; vì vậy chuyển sang tính **%yes theo các phân khúc** (age band, job/education, marital/housing/loan/default) rồi huấn luyện **Logistic (One-Hot + scale age, class_weight='balanced')** để xếp hạng khách hàng.


## Phần 3: EDA — Conversion by Segment (3 insight)
- Tính **%yes** theo các phân khúc:
  1) Nhân khẩu học: `age_band`, `job`, `education`
  2) Gia đình: `marital`
  3) Tài chính/rủi ro: `housing`, `loan`, `default`


In [12]:
def conversion_table(df_in, group_col, y_col="y_bin", min_support=0):
    g = (
        df_in.groupby(group_col, dropna=False)[y_col]
        .agg(total="count", yes_count="sum")
        .reset_index()
    )
    g["yes_ratio (%)"] = (g["yes_count"] / g["total"] * 100).round(2)
    g = g.sort_values(["yes_ratio (%)", "total"], ascending=[False, False])
    if min_support > 0:
        g = g[g["total"] >= min_support]
    return g


In [13]:
# Tạo age band: ≤25, 26–35, 36–50, >50
age_bins = [-1, 25, 35, 50, 200]
age_labels = ["≤25", "26–35", "36–50", ">50"]
df_personal["age_band"] = pd.cut(df_personal["age"], bins=age_bins, labels=age_labels)

conv_age = conversion_table(df_personal, "age_band")
display(conv_age)


/tmp/ipykernel_20088/2140825068.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_in.groupby(group_col, dropna=False)[y_col]


,age_band,total,yes_count,yes_ratio (%)
0,≤25,1666,349,20.95
3,>50,7180,1082,15.07
1,26–35,14847,1740,11.72
2,36–50,17495,1469,8.40


### Nhận xét — %yes theo **age_band**

**Kết quả nổi bật**

* **≤25: 20.95%** (349/1,666) — **cao nhất**, nhưng quy mô nhỏ.
* **>50: 15.07%** (1,082/7,180) — tỷ lệ tốt và **quy mô khá lớn**.
* **26–35: 11.72%** (1,740/14,847) — gần mức chung (≈11.27%) nhưng **đem lại nhiều “yes” tuyệt đối nhất**.
* **36–50: 8.40%** (1,469/17,495) — **thấp nhất**, lại là nhóm đông nhất.

**Ý nghĩa/So what**

* Nếu tối ưu **tỷ lệ**: ưu tiên **≤25** và **>50**.
* Nếu tối ưu **số lượng “yes” tuyệt đối**: đẩy mạnh **26–35** (nền khách lớn).
* Nhóm **36–50** nên **điều chỉnh kịch bản** (nhấn mạnh an toàn/lợi suất/ưu đãi) hoặc kết hợp lọc thêm theo **job/education** để tìm “pocket” tiềm năng.

**Lưu ý**

* Kiểm tra **ý nghĩa thống kê** giữa các band (z-test tỉ lệ) trước khi chốt phân bổ.
* Có thể có **tương tác** với nghề/học vấn hoặc mùa vụ; đừng quyết định chỉ theo tuổi.

**Hành động đề xuất**

* Chạy thêm bảng **%yes theo (age_band × job)** và **(age_band × education)**.
* Chuẩn bị 2 kịch bản telesales:

  * **Trẻ (≤25)**: nhấn mạnh **mục tiêu tài chính sớm/ưu đãi**.
  * **Lớn tuổi (>50)**: nhấn **an toàn/lãi suất ổn định**.
* Với **36–50**, thử **thời điểm gọi/kênh** và ưu đãi phù hợp trước khi giảm ưu tiên.


In [17]:
conv_job = conversion_table(df_personal, "job")
conv_edu = conversion_table(df_personal, "education")

print("Conversion by job:")
display(conv_job)

print("Conversion by education:")
display(conv_edu)


Conversion by job:


,job,total,yes_count,yes_ratio (%)
8,student,875,275,31.43
5,retired,1720,434,25.23
10,unemployed,1014,144,14.20
0,admin.,10422,1352,12.97
4,management,2924,328,11.22
11,unknown,330,37,11.21
9,technician,6743,730,10.83
6,self-employed,1421,149,10.49
3,housemaid,1060,106,10.00
2,entrepreneur,1456,124,8.52


Conversion by education:


,education,total,yes_count,yes_ratio (%)
4,illiterate,18,4,22.22
7,unknown,1731,251,14.50
6,university.degree,12168,1670,13.72
5,professional.course,5243,595,11.35
3,high.school,9515,1031,10.84
0,basic.4y,4176,428,10.25
1,basic.6y,2292,188,8.20
2,basic.9y,6045,473,7.82


### Nhận xét — Conversion theo **job** & **education**

**Job**
- **Cao nhất:** `student` **31.43%** (N=875) và `retired` **25.23%** (N=1,720) → tỷ lệ rất tốt; `retired` có quy mô khá.
- **Quy mô lớn, tỷ lệ ổn:** `admin.` **12.97%** (N=10,422) → đóng góp **nhiều “yes” tuyệt đối**; `technician` **10.83%** (N=6,743).
- **Thấp nhất:** `blue-collar` **6.89%** (N=9,254) → cần **kịch bản riêng** hoặc **giảm ưu tiên** nếu tối ưu tỷ lệ.
- **Khác:** `unemployed` **14.20%** (N=1,014) đáng chú ý; `services` **8.14%** (N=3,969).

**Education**
- **Cao nhưng nhỏ mẫu:** `illiterate` **22.22%** (N=18) → **không kết luận** từ mẫu quá nhỏ.
- **Mục tiêu tốt & lớn mẫu:** `university.degree` **13.72%** (N=12,168) → vừa **tỷ lệ tốt** vừa **quy mô lớn** → nhóm **ưu tiên**.
- `unknown` **14.50%** (N=1,731) cao hơn trung bình → **giữ tách riêng** khi mô hình hóa.
- **Thấp:** `basic.9y` **7.82%** (N=6,045), `basic.6y` **8.20%** → cân nhắc **giảm ưu tiên** hoặc cá nhân hóa ưu đãi.

**So what**
- Nếu tối ưu **tỷ lệ**: tập trung `student`, `retired`, `unemployed` và `university.degree`.
- Nếu tối ưu **số lượng “yes”**: `admin.` và `university.degree` vì **nền khách lớn**.
- `blue-collar` và nhóm **basic.*y** cần **điều chỉnh kịch bản** (nhấn lợi ích ngắn hạn/ưu đãi phí) hoặc **giảm tần suất**.

**Lưu ý**
- Kiểm tra **ý nghĩa thống kê** khác biệt giữa nhóm (z-test tỉ lệ).
- Tránh thiên lệch do **mùa vụ/kênh liên hệ**; sẽ kiểm tra thêm giao cắt.

**Hành động kế tiếp**
- Tạo bảng **(age_band × job)** và **(education × job)** để tìm “pocket” có **%yes cao + đủ mẫu** cho chiến dịch ưu tiên.


In [18]:
conv_marital = conversion_table(df_personal, "marital")
conv_housing = conversion_table(df_personal, "housing")
conv_loan = conversion_table(df_personal, "loan")
conv_default = conversion_table(df_personal, "default")

print("Conversion by marital:")
display(conv_marital)

print("Conversion by housing:")
display(conv_housing)

print("Conversion by loan:")
display(conv_loan)

print("Conversion by default (giữ 'unknown'):")
display(conv_default)


Conversion by marital:


,marital,total,yes_count,yes_ratio (%)
3,unknown,80,12,15.00
2,single,11568,1620,14.00
0,divorced,4612,476,10.32
1,married,24928,2532,10.16


Conversion by housing:


,housing,total,yes_count,yes_ratio (%)
2,yes,21576,2507,11.62
0,no,18622,2026,10.88
1,unknown,990,107,10.81


Conversion by loan:


,loan,total,yes_count,yes_ratio (%)
0,no,33950,3850,11.34
2,yes,6248,683,10.93
1,unknown,990,107,10.81


Conversion by default (giữ 'unknown'):


,default,total,yes_count,yes_ratio (%)
0,no,32588,4197,12.88
1,unknown,8597,443,5.15
2,yes,3,0,0.00


### Nhận xét — Conversion theo **marital / housing / loan / default**

**Marital**
- **single 14.00%** (N=11,568) > divorced 10.32% > married **10.16%**.  
  → **Ưu tiên nhóm single** (tỷ lệ cao hơn rõ rệt). `unknown 15%` nhưng N=80 (quá nhỏ).

**Housing**
- `yes` **11.62%** (N=21,576) nhỉnh hơn `no` **10.88%**; `unknown` **10.81%**.  
  → Khác biệt **nhẹ**, **không đủ** để tách nhóm một mình; nên kết hợp với **age/job**.

**Loan**
- `no` **11.34%** ≈ `yes` **10.93%** (chênh không đáng kể).  
  → **Ít giá trị phân tách** nếu đứng riêng.

**Default (giữ 'unknown')**
- `no` **12.88%** (N=32,588) **>>** `unknown` **5.15%** (N=8,597); `yes` N=3 (0%).  
  → **Tín hiệu mạnh**: ưu tiên khách **default = no**; nhóm **default = unknown** nên **giảm ưu tiên** hoặc cần **kịch bản nuôi dưỡng**.

**So what (ngắn gọn)**
- **Ưu tiên gọi:** `single` + `default=no`.  
- **Cần kết hợp thêm điều kiện:** `housing`/`loan` (tự chúng phân tách yếu).  
- **Tiếp theo:** kiểm tra giao cắt **(age_band × marital)** và **(default × job/education)** để xác định các “pocket” %yes cao + đủ mẫu.
